In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [7]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
import os
from langchain_openai import ChatOpenAI  # Notez le changement ici

# Définition de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-0xUZ6aBpi14QWLtzQC2nF0B2gQTojxukve0byW1qgx0 8mq8WeoK0yCT3BlbkFJ FfHS_cUMhXQMX6_U0pycw_XiZUUtZ4V6Gc5xEwhMZOsYA6xKN4HruNnPRcA"

# Création du modèle
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Notez que c'est 'model' et non 'model_name' dans les versions récentes
    temperature=0
)

# One-Shot AutoDiCot + Full Context

One-Shot AutoDiCot + Full Context, explored by prompt engineers, is an adaptive technique that enhances language model (LLM) performance by identifying and correcting specific misclassifications.

## How Does One-Shot AutoDiCot + Full Context Work?

Imagine you're coaching a talented student who occasionally misunderstands certain problem types. Instead of revamping your entire teaching approach, you could be more efficient by pinpointing those specific misconceptions and addressing them directly.

One-Shot AutoDiCot + Full Context follows this logic in three key steps:

1. **Error Identification**: After performing 10-shot prompting, a specific instance (like the 12th item in the development set) that's being incorrectly classified is identified.

2. **Explanation Generation**: The LLM is prompted to generate an explanation of why this mislabeling occurred, revealing potential reasoning flaws or biases in how the model interprets the example.

3. **Targeted Refinement**: Based on this explanation, the prompt is carefully modified to correct the specific error while maintaining performance on other examples. This might involve adding clarifying examples or emphasizing overlooked features relevant to the classification.

In [9]:
one_shot_autodicot_full_context_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate solutions using One-Shot AutoDiCot + Full Context for a given classification problem.

    One-Shot AutoDiCot + Full Context involves analyzing incorrect classifications from a 10-shot prompt, prompting the LLM to explain the mislabeling, and then modifying the prompt to correct the error.

    Please structure your response in the following format:

    Step 1: Identify Mislabeling
    [Identify a specific mislabeled instance from a 10-shot prompt.]

    Step 2: Generate Explanation
    [Prompt the LLM to explain why the instance was mislabeled.]

    Step 3: Modify Prompt
    [Modify the prompt based on the explanation to correct the mislabeling.]

    Step 4: Verify Correction
    [Verify that the modified prompt correctly labels the instance.]

    Step 5: Final Solution
    [Provide the corrected classification and explanation.]

    Example:
    Problem: Classify sentiment as positive or negative.
    10-Shot Prompt: [Examples of positive and negative sentiment classifications]
    Instance 12: "This movie was surprisingly boring." (Incorrectly labeled as positive)

    Step 1: Identify Mislabeling
    Instance 12: "This movie was surprisingly boring." (Incorrectly labeled as positive)

    Step 2: Generate Explanation
    Explanation: The LLM might have focused on "surprisingly" and interpreted it as a positive surprise, ignoring "boring."

    Step 3: Modify Prompt
    Modified Prompt: [10-shot examples with emphasis on negative keywords like "boring" and "disappointing"]
    Include an explicit example: "A movie that is boring is negative."

    Step 4: Verify Correction
    The modified prompt correctly labels "This movie was surprisingly boring" as negative.

    Step 5: Final Solution
    Corrected Classification: Negative.
    Explanation: The modified prompt clarifies the importance of negative keywords in sentiment classification.
    """),
    ("human", "{problem_description}"),
    ("assistant", """
    I will use One-Shot AutoDiCot + Full Context to correct a mislabeling.

    Step 1: Identify Mislabeling
    [Identify a mislabeled instance.]

    Step 2: Generate Explanation
    [Generate an explanation for the mislabeling.]

    Step 3: Modify Prompt
    [Modify the prompt to correct the mislabeling.]

    Step 4: Verify Correction
    [Verify the correction.]

    Step 5: Final Solution
    [Provide the corrected classification and explanation.]
    """)
])

# Most Suitable Questions for One-Shot AutoDiCot + Full Context

For One-Shot AutoDiCot + Full Context, the most suitable questions are those where identifying and correcting specific misclassifications can significantly enhance model performance. Here are types of questions that would be particularly well-suited to this approach:

### Classification Tasks with Edge Cases
- Sentiment analysis with mixed or subtle emotions
- Spam detection with legitimate-looking marketing content
- Toxicity classification with cultural nuances
- Topic categorization with overlapping themes

### Contextual Misinterpretation Scenarios
- Intent classification with ambiguous phrasing
- Sarcasm or irony detection in text
- Emergency vs. non-emergency identification in messages
- Professional vs. casual language classification

### Pattern Recognition Challenges
- Questions where conflicting signal words appear
- Classification tasks with misleading features
- Cases where exceptions to general rules apply
- Scenarios where word order significantly alters meaning

### Concrete Examples of Questions

1. "Classify whether this text expresses positive or negative sentiment: 'The movie was surprisingly boring but the soundtrack was excellent.'"
   - Contains mixed signals that might confuse classification

2. "Determine if this email is spam: 'Important notification: Your recent order has been processed. Click here to view shipping details.'"
   - Contains legitimate-looking content with potential spam indicators

3. "Categorize this news headline as factual reporting or opinion: 'Studies suggest economy may face challenges in coming months'"
   - Contains qualifying language that makes classification challenging

4. "Classify this customer feedback as a complaint or suggestion: 'Your product would be better if it had the features your competitors offer'"
   - Indirect criticism that could be misclassified

5. "Identify whether this social media post contains misinformation: 'Scientists have different theories about the optimal diet for weight loss'"
   - Contains truthful statements that could be misinterpreted

One-Shot AutoDiCot + Full Context is particularly effective for these types of questions because:

- It identifies specific instances where misclassification occurs rather than general patterns
- It generates explanations for why the misclassification happened, revealing model reasoning flaws
- It allows for targeted prompt modifications to address specific error cases
- It improves performance by focusing on challenging edge cases while maintaining accuracy on clearer examples
- It provides insight into model reasoning processes that can inform broader prompt engineering

This approach is especially valuable in classification contexts where subtle features or conflicting signals can lead to misclassifications, and where understanding the specific reasoning failures can lead to more robust prompt design.

In [10]:
import pandas as pd
import re
from typing import Optional, TypedDict, Annotated
from langchain.schema import AIMessage, HumanMessage

# Function to generate the One-Shot AutoDiCot + Full Context prompt and extract the final answer (Step 5)
def generate_one_shot_autodicot_node(state):
    question = state['messages'][-1].content  # Get the last question
    prompt_value = one_shot_autodicot_full_context_template.invoke({"problem_description": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    
    # Keep the full response for processing
    full_response = response.content
    
    # Extract the final solution from Step 5
    final_solution_pattern = r'Step 5:([\s\S]*?)(?=$|Step \d:)'
    final_solution_match = re.search(final_solution_pattern, full_response)
    
    if final_solution_match:
        final_solution = final_solution_match.group(1).strip()
    else:
        # Alternative pattern
        alternative_pattern = r'Step 5: \[Final Solution\]([\s\S]*?)(?=$|Step \d:)'
        alternative_match = re.search(alternative_pattern, full_response)
        
        if alternative_match:
            final_solution = alternative_match.group(1).strip()
        else:
            final_solution = "Final solution not found in the expected format."
    
    # Save the question, final solution, and full processing to a text file
    with open('one_shot_autodicot_result.txt', 'w', encoding='utf-8') as f:
        f.write("=== PROBLEM DESCRIPTION ===\n\n")
        f.write(question)
        f.write("\n\n=== FINAL SOLUTION (STEP 5) ===\n\n")
        f.write(final_solution)
        f.write("\n\n=== COMPLETE PROCESSING (FOR REFERENCE) ===\n\n")
        f.write(full_response)
    
    # Return only the final solution in messages, plus the original question for reference
    return {
        "messages": [AIMessage(content=final_solution)],
        "problem_description": question,
        "full_response": full_response
    }

# Definition of state class
class State(TypedDict):
    messages: Annotated[list, add_messages]
    problem_description: Optional[str]
    full_response: Optional[str]

# Graph creation
graph_builder = StateGraph(State)
graph_builder.add_node("generate_one_shot_autodicot", generate_one_shot_autodicot_node)

# Graph configuration
graph_builder.set_entry_point("generate_one_shot_autodicot")
graph_builder.add_edge("generate_one_shot_autodicot", END)
graph = graph_builder.compile()

# Usage example
inputs = {"messages": [HumanMessage(content="Classify the following email as spam or not spam: "
                                        "'CONGRATULATIONS! You've been selected to receive $10,000,000 from "
                                        "a foreign prince. Reply with your bank details to claim your prize.'")],
          "result_df": None}  # Initialize result_df to None
 
# Print the problem description at the beginning
print("PROBLEM DESCRIPTION:")
print(inputs["messages"][0].content)
print("\nFINAL SOLUTION (STEP 5):")

# Graph execution - only display Step 5
original_problem = ""
for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_one_shot_autodicot":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)
                    
            # Store problem description for reference
            if "problem_description" in value:
                original_problem = value["problem_description"]

# Print confirmation that results were saved
print("\n(Complete results saved in 'one_shot_autodicot_result.txt')")

PROBLEM DESCRIPTION:
Classify the following email as spam or not spam: 'CONGRATULATIONS! You've been selected to receive $10,000,000 from a foreign prince. Reply with your bank details to claim your prize.'

FINAL SOLUTION (STEP 5):
Final Solution  
Corrected Classification: Spam.  
Explanation: The modified prompt emphasizes the characteristics of spam emails, such as unsolicited financial offers and requests for sensitive information, leading to the correct classification of the email as spam.

(Complete results saved in 'one_shot_autodicot_result.txt')
